In [22]:
import gym
import ray

In [23]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [24]:
# Initialize Ray
ray.shutdown()

In [4]:
from rayflow import Rayflow
r = Rayflow.load("test_input.yml", minio_volume_path="./minio_volume/", mlflow_volume_path="./mlflow_volume/",no_ray=True)

HOST INFO {'192.168.1.65': {'ansible_python_interpreter': '~/anaconda3/envs/py37/bin/python', 'ansible_connection': 'local'}}


/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (127.0.0.1) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


In [5]:
r.start()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (192.168.1.65) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-ke

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-82606j0ai1u80/tmp6w7sq138 TO /home/shyam/.ansible/tmp/ansible-tmp-1601340310.1257794-25247611357468/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601340310.1257794-25247611357468/ /home/shyam/.ansible/tmp/ansible-tmp-1601340310.1257794-25247611357468/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601340310.1257794-25247611357468/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601340310.1257794-25247611357468/ > /dev/null 2>&1 && sleep 0'
ok: [192.168.1.65] => {
    "changed": false,
    "invocation": {
        "module_args": {
            "api_version": "auto",
         

INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-82799dx8hmaiu/tmpx1l_5exm TO /home/shyam/.ansible/tmp/ansible-tmp-1601340314.373487-148113240885723/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601340314.373487-148113240885723/ /home/shyam/.ansible/tmp/ansible-tmp-1601340314.373487-148113240885723/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601340314.373487-148113240885723/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601340314.373487-148113240885723/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",
    

<192.168.1.65> EXEC /bin/sh -c '( umask 77 && mkdir -p "` echo /home/shyam/.ansible/tmp/ansible-tmp-1601340315.2045429-83602817388227 `" && echo ansible-tmp-1601340315.2045429-83602817388227="` echo /home/shyam/.ansible/tmp/ansible-tmp-1601340315.2045429-83602817388227 `" ) && sleep 0'
Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-82799dx8hmaiu/tmpm2gk4a64 TO /home/shyam/.ansible/tmp/ansible-tmp-1601340315.2045429-83602817388227/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601340315.2045429-83602817388227/ /home/shyam/.ansible/tmp/ansible-tmp-1601340315.2045429-83602817388227/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601340315.2045429-83602817388227/AnsiballZ_docker_container.py &&

2020-09-28 17:45:15,959	INFO resource_spec.py:231 -- Starting Ray with 32.32 GiB memory available for workers and up to 16.18 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601340315.2045429-83602817388227/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",
            "Args": [
                "server",
                "--backend-store-uri",
                "/mlflow",
                "--host",
                "192.168.1.65",
                "-p",
                "5000",
                "--default-artifact-root",
                "s3://artifacts/"
            ],
            "Config": {
                "AttachStderr": false,
                "AttachStdin": false,
                "AttachStdout": false,
                "Cmd": [
                    "mlflow",
                    "server",
                    "--backend-store-uri",
                    "/mlflow",
                    "--host",
                    "192.168.1.65",
                    "-p",
                    "5000"

2020-09-28 17:45:16,350	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


In [6]:
from minerl_rllib.envs import register
register()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [18]:
from ray.rllib.agents.ppo import PPOTrainer

train_name = "mlflow_train"
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"JESUS",
    "create_new_experiment": True
}
trainer_config = {
        "env": "MineRLNavigateDenseVectorObf-v0",
         "num_gpus": 1,
         "num_workers": 1,
         "num_envs_per_worker": 1,
         "eager": False,
         "use_pytorch": True,
         'monitor':True, 
}
saver_config = {
        "checkpoint_on_end": True,
        "checkpoint_freq": 2
}
stop = {
        "training_iteration": 3,
}
tune_args = {
        "checkpoint_at_end": True,
        "checkpoint_freq": 2
}

config = {
    "trainer_config":trainer_config,
    "logging_config": mlflow_logging_config
}

In [19]:
from rayflow.tune import Tune
t = Tune()

In [20]:
import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer

In [21]:
run_id, output_config, experiment_analysis = t.run_training(
               trainer=PPOTrainer, 
               name=train_name, 
               stop=stop, 
               mlflow_logging_config=mlflow_logging_config, 
               trainer_config=trainer_config,
               saver_config=saver_config,
               tune_args=tune_args,
               )

ConnectionError: HTTPConnectionPool(host='192.168.1.65', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f957ca25208>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
r.stop()

### Register MineRL Gym Environment to RLlib

In [ ]:
from minerl_rllib.envs import register

In [ ]:
register()

In [ ]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples
# config = ppo.DEFAULT_CONFIG.copy()
# config["num_gpus"] = 1
# config["num_workers"] = 1
# config["num_envs_per_worker"] = 1
# config["eager"] = False
# trainer = ppo.PPOTrainer(config=config, env="MineRLNavigateDenseVectorObf-v0")


tune.run(PPOTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
#                  'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 3})  

In [15]:
r.stop()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (192.168.1.65) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-85689guav5glb/tmp4q1kpqzh TO /home/shyam/.ansible/tmp/ansible-tmp-1601340707.889693-81126319892759/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601340707.889693-81126319892759/ /home/shyam/.ansible/tmp/ansible-tmp-1601340707.889693-81126319892759/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601340707.889693-81126319892759/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601340707.889693-81126319892759/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "changed": true,
    "invocation": {
        "module_args": {
            "api_version": "auto",
          